In [1]:
import numpy as np
import pandas as pd
import csv
import random
import math

In [2]:
df = pd.read_csv('data/titanik.csv', sep=',')

drop_column = ['SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin']

# подготовленный датасет
train_data = []
# датасет для тестирования
test_data = []

classes = df['Survived'].unique()

df['Sex'] = df['Sex'].replace(['male','female'], [0, 1])
df['Embarked'] = df['Embarked'].fillna(('S'))
df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

df['Pclass'] = df['Pclass'].fillna(np.mean(df['Pclass']))
df['Pclass'] = df['Pclass']/max(df['Pclass'])

df['Age'] = df['Age'].fillna(df['Age'].median())
df['Age'] = df['Age']/max(df['Age'])


for i in range(len(df)) : 

    features = np.zeros((4))
    features[0] = df.loc[i, 'Sex']
    features[1] = df.loc[i, 'Embarked']
    features[2] = df.loc[i, 'Pclass']
    features[3] = df.loc[i, 'Age']
    
    
    # Нормализация входного вектора
    norm_features = np.zeros((4))
    for j in range(len(features)):
        sum = 0 
        for k in range(len(features)):
            sum += features[k] ** 2
        norm_features[j] = features[j] / math.sqrt(sum)
    
    # Нормализованные    
    temp_data = dict(name = df.loc[i, 'Name'], 
                     features = norm_features, 
                     result = np.where(classes == df.loc[i, "Survived"])[0])
    
    # Не нормализованные    
#     temp_data = dict(name = df.loc[i, 'Name'], 
#                      features = features, 
#                      result = np.where(classes == df.loc[i, "Survived"])[0])
    
    if (len(train_data) / len(df)) < 0.9:
        train_data.append(temp_data)
    else:
        test_data.append(temp_data)

In [3]:
# train_data = []
# test_data = []
# alice_data = dict(name = 'Alice', 
#                      features = np.array([-2, -1]), 
#                      result = [1, 0])
# bob_data = dict(name = 'Bob', 
#                      features = np.array([25, 6]), 
#                      result = [0, 1])
# charlie_data = dict(name = 'Charlie', 
#                      features = np.array([17, 4]), 
#                      result = [0, 1])
# diana_data = dict(name = 'Diana', 
#                      features = np.array([-15, -6]), 
#                      result = [1, 0])
# jack_data = dict(name = 'Jack', 
#                      features = np.array([11, 5]), 
#                      result = [0, 1])

# train_data.append(alice_data)
# train_data.append(bob_data)
# train_data.append(charlie_data)
# train_data.append(diana_data)

# test_data.append(jack_data)
# test_data.append(alice_data)
# test_data.append(bob_data)
# test_data.append(charlie_data)
# test_data.append(diana_data)

In [4]:
def activation(net):
    res = 1.0/(1 + np.exp(-net))
    return res
def logistic_deriv(x):
    return activation(x) * (1 - activation(x))
def mse(pred, actual):
    return np.square(np.subtract(actual,pred)).mean()

In [5]:
# Обучение перцептрона
epoch_count = 30
per_input_neuron_count = len(train_data[0]['features'])
hiden_layer_count = 2
output_neuron_count = len(train_data[0]['result'])
# количество нейронов в слоях, последний слой выходной, первый - входной
neuron_count = [per_input_neuron_count, 10, 5, output_neuron_count]
max_neuron_count = np.max(neuron_count)

#инициализация начальных весов Перцептрона weights[l][x][y]
# l - индекс слоя (слои считаются с первого скрытого слоя)
# x - индекс нейрона от которого идёт вес
# y - индекс нейрона к которому идёт вес
weights = np.random.uniform(0, 1, (len(neuron_count) - 1, max_neuron_count, max_neuron_count))
new_weights = np.zeros((len(neuron_count) - 1, max_neuron_count, max_neuron_count))

step = epoch_count * len(train_data)

# до активации нейронов preActiv[x][y]. Начинает с первого скрытого слоя
# x - индекс слоя (слои считаются с первого скрытого слоя)
# y - индекс нейрона в слое
preActiv = np.zeros((len(neuron_count) - 1, max_neuron_count))
# после активациии нейронов postActiv[x][y]
# x - индекс слоя (слои считаются с входного слоя)
# y - индекс нейрона в слое
postActiv = np.zeros((len(neuron_count), max_neuron_count))

# ошибка на нейронах скрытых слоев
err = np.zeros((len(neuron_count) - 1, np.max(neuron_count[1:len(neuron_count)-1])))

# скорость обучения
theta = 0.1

# Обучение на подготовленном датасете
for ep in range(epoch_count):
    for el in range(len(train_data)):
    
        # получаем нормализованный входной вектор со слоя Кохонена
        # student = train_data[random.randint(0, len(train_data) - 1)]
        # train_el = getKohonenNormResult(student['features'], kohonen_neuron_count, kohonen_w)

        # подаем на вход датасет
        student = train_data[el]
        train_el = student['features']

        for i in range(len(train_el)):
            postActiv[0][i] = train_el[i]

        # Проходим по всем слоям
        for l in range(len(neuron_count) - 1):
            for n_c in range(neuron_count[l + 1]):
                preActiv[l][n_c] = np.dot(postActiv[l][0:neuron_count[l]], weights[l][0:neuron_count[l],n_c])
                postActiv[l+1][n_c] = activation(preActiv[l][n_c])

        # Вычисляем ошибку на выходе
        for i in range(len(student['result'])):
            err[len(err) - 1][i] = student['result'][i] - postActiv[len(neuron_count)-1][i]

        # Вычисляем ошибку на скрытых слоях
        for l in range(len(neuron_count) - 2, 0, -1):
            for n_c in range(neuron_count[l]):
                err[l - 1][n_c] = np.dot(err[l][0:neuron_count[l + 1]], weights[l][n_c,0:neuron_count[l + 1]])

        # Находим новые веса
        for l in range(len(neuron_count) - 1):
            for n_c in range(neuron_count[l + 1]):
                temp = theta * err[l][n_c] * logistic_deriv(preActiv[l][n_c])
                for in_c in range(neuron_count[l]):
                    new_weights[l][in_c, n_c] = postActiv[l][in_c] * temp + weights[l][in_c, n_c]

        weights = new_weights.copy()

        # Изменение скорости обучения
    #     print('==============')
        theta -= (theta * 0.0002)
        print(theta)
        step -= 1

0.09998
0.099960004
0.09994001199920001
0.09992002399680017
0.09990003999200081
0.09988005998400241
0.09986008397200562
0.09984011195521121
0.09982014393282017
0.0998001799040336
0.0997802198680528
0.0997602638240792
0.09974031177131439
0.09972036370896012
0.09970041963621833
0.09968047955229108
0.09966054345638062
0.09964061134768934
0.0996206832254198
0.0996007590887747
0.09958083893695695
0.09956092276916956
0.09954101058461573
0.09952110238249881
0.09950119816202231
0.09948129792238991
0.09946140166280543
0.09944150938247287
0.09942162108059638
0.09940173675638025
0.09938185640902898
0.09936198003774718
0.09934210764173963
0.09932223922021129
0.09930237477236724
0.09928251429741276
0.09926265779455327
0.09924280526299437
0.09922295670194177
0.09920311211060139
0.09918327148817926
0.09916343483388163
0.09914360214691485
0.09912377342648547
0.09910394867180017
0.0990841278820658
0.09906431105648938
0.09904449819427809
0.09902468929463923
0.0990048843567803
0.09898508337990894
0.09896

0.09041047229354586
0.09039239019908715
0.09037431172104733
0.09035623685870313
0.09033816561133139
0.09032009797820913
0.09030203395861348
0.09028397355182176
0.09026591675711139
0.09024786357375997
0.09022981400104521
0.090211768038245
0.09019372568463735
0.09017568693950043
0.09015765180211252
0.0901396202717521
0.09012159234769775
0.09010356802922821
0.09008554731562236
0.09006753020615924
0.090049516700118
0.09003150679677797
0.09001350049541862
0.08999549779531954
0.08997749869576048
0.08995950319602133
0.08994151129538212
0.08992352299312305
0.08990553828852442
0.08988755718086672
0.08986957966943054
0.08985160575349666
0.08983363543234596
0.08981566870525949
0.08979770557151844
0.08977974603040413
0.08976179008119806
0.08974383772318181
0.08972588895563717
0.08970794377784604
0.08969000218909047
0.08967206418865266
0.08965412977581493
0.08963619894985976
0.0896182717100698
0.08960034805572778
0.08958242798611664
0.08956451150051942
0.08954659859821931
0.08952868927849966
0.0895

0.08162616689358206
0.08160984166020334
0.0815935196918713
0.08157720098793292
0.08156088554773533
0.08154457337062578
0.08152826445595165
0.08151195880306046
0.08149565641129985
0.0814793572800176
0.08146306140856159
0.08144676879627988
0.08143047944252062
0.08141419334663211
0.08139791050796279
0.08138163092586119
0.08136535459967602
0.08134908152875608
0.08133281171245034
0.08131654515010785
0.08130028184107782
0.0812840217847096
0.08126776498035265
0.08125151142735658
0.08123526112507111
0.0812190140728461
0.08120277027003153
0.08118652971597752
0.08117029241003433
0.08115405835155233
0.08113782753988202
0.08112159997437404
0.08110537565437917
0.0810891545792483
0.08107293674833245
0.08105672216098278
0.08104051081655059
0.08102430271438728
0.0810080978538444
0.08099189623427364
0.08097569785502678
0.08095950271545578
0.08094331081491268
0.0809271221527497
0.08091093672831916
0.08089475454097349
0.0808785755900653
0.08086239987494728
0.0808462273949723
0.0808300581494933
0.08081389

0.07235161838596944
0.07233714806229224
0.07232268063267978
0.07230821609655325
0.07229375445333394
0.07227929570244328
0.07226483984330279
0.07225038687533412
0.07223593679795906
0.07222148961059947
0.07220704531267735
0.07219260390361482
0.07217816538283409
0.07216372974975752
0.07214929700380757
0.07213486714440681
0.07212044017097793
0.07210601608294374
0.07209159487972716
0.07207717656075122
0.07206276112543906
0.07204834857321397
0.07203393890349934
0.07201953211571864
0.0720051282092955
0.07199072718365364
0.07197632903821691
0.07196193377240927
0.07194754138565479
0.07193315187737766
0.07191876524700218
0.07190438149395277
0.07189000061765398
0.07187562261753044
0.07186124749300694
0.07184687524350834
0.07183250586845964
0.07181813936728595
0.0718037757394125
0.07178941498426462
0.07177505710126778
0.07176070208984753
0.07174634994942956
0.07173200067943968
0.0717176542793038
0.07170331074844793
0.07168897008629825
0.071674632292281
0.07166029736582254
0.07164596530634937
0.071

0.06447814312771087
0.06446524749908533
0.06445235444958551
0.0644394639786956
0.06442657608589986
0.06441369077068268
0.06440080803252854
0.06438792787092204
0.06437505028534786
0.06436217527529078
0.06434930284023573
0.06433643297966768
0.06432356569307175
0.06431070097993313
0.06429783883973715
0.0642849792719692
0.06427212227611481
0.06425926785165958
0.06424641599808925
0.06423356671488964
0.06422072000154666
0.06420787585754635
0.06419503428237484
0.06418219527551837
0.06416935883646327
0.06415652496469598
0.06414369365970304
0.0641308649209711
0.06411803874798691
0.06410521514023731
0.06409239409720927
0.06407957561838983
0.06406675970326614
0.06405394635132548
0.06404113556205522
0.0640283273349428
0.06401552166947581
0.06400271856514192
0.0639899180214289
0.0639771200378246
0.06396432461381704
0.06395153174889427
0.0639387414425445
0.06392595369425598
0.06391316850351712
0.06390038586981642
0.06388760579264245
0.06387482827148393
0.06386205330582963
0.06384928089516846
0.06383

...

0.0013304779098313605
0.0013302118142493942
0.0013299457718865444
0.0013296797827321671
0.0013294138467756208
0.0013291479640062656
0.0013288821344134643
0.0013286163579865815
0.0013283506347149842
0.0013280849645880412
0.0013278193475951235
0.0013275537837256046
0.0013272882729688594
0.0013270228153142655
0.0013267574107512026
0.0013264920592690524
0.0013262267608571987
0.0013259615155050272
0.0013256963232019263
0.001325431183937286
0.0013251660977004985
0.0013249010644809584
0.0013246360842680623
0.0013243711570512086
0.0013241062828197984
0.0013238414615632345
0.001323576693270922
0.0013233119779322677
0.0013230473155366811
0.0013227827060735737
0.0013225181495323591
0.0013222536459024526
0.001321989195173272
0.0013217247973342373
0.0013214604523747705
0.0013211961602842956
0.0013209319210522386
0.0013206677346680282
0.0013204036011210947
0.0013201395204008705
0.0013198754924967904
0.0013196115173982911
0.0013193475950948115
0.0013190837255757926
0.0013188199088306773
0.00131855614

0.0011396517420735092
0.0011394238117250946
0.0011391959269627496
0.001138968087777357
0.0011387402941598014
0.0011385125461009695
0.0011382848435917492
0.0011380571866230309
0.0011378295751857063
0.0011376020092706692
0.001137374488868815
0.0011371470139710414
0.0011369195845682472
0.0011366922006513336
0.0011364648622112033
0.001136237569238761
0.0011360103217249132
0.0011357831196605681
0.001135555963036636
0.0011353288518440286
0.0011351017860736598
0.001134874765716445
0.0011346477907633018
0.001134420861205149
0.001134193977032908
0.0011339671382375016
0.0011337403448098541
0.0011335135967408922
0.0011332868940215441
0.0011330602366427399
0.0011328336245954114
0.0011326070578704924
0.0011323805364589182
0.0011321540603516265
0.0011319276295395561
0.0011317012440136482
0.0011314749037648454
0.0011312486087840924
0.0011310223590623356
0.0011307961545905232
0.0011305699953596052
0.0011303438813605332
0.001130117812584261
0.0011298917890217443
0.00112966581066394
0.001129439877501807

0.0010451015577777317
0.0010448925374661761
0.001044683558958683
0.0010444746222468913
0.0010442657273224418
0.0010440568741769774
0.001043848062802142
0.0010436392931895816
0.0010434305653309437
0.0010432218792178775
0.001043013234842034
0.0010428046321950657
0.0010425960712686267
0.001042387552054373
0.0010421790745439622
0.0010419706387290535
0.0010417622446013076
0.0010415538921523873
0.0010413455813739567
0.0010411373122576819
0.0010409290847952304
0.0010407208989782714
0.0010405127547984756
0.001040304652247516
0.0010400965913170664
0.001039888571998803
0.0010396805942844032
0.0010394726581655464
0.0010392647636339133
0.0010390569106811865
0.0010388490992990502
0.0010386413294791903
0.0010384336012132945
0.0010382259144930519
0.0010380182693101533
0.0010378106656562913
0.00103760310352316
0.0010373955829024555
0.001037188103785875
0.0010369806661651177
0.0010367732700318846
0.0010365659153778782
0.0010363586021948027
0.0010361513304743637
0.0010359441002082688
0.00103573691138822

0.0009593546156430086
0.00095916274471988
0.0009589709121709361
0.0009587791179885019
0.0009585873621649042
0.0009583956446924712
0.0009582039655635327
0.00095801232477042
0.0009578207223054659
0.0009576291581610049
0.0009574376323293727
0.0009572461448029068
0.0009570546955739462
0.0009568632846348315
0.0009566719119779045
0.000956480577595509
0.0009562892814799898
0.0009560980236236939
0.0009559068040189691
0.0009557156226581653
0.0009555244795336336
0.0009553333746377269
0.0009551423079627993
0.0009549512795012068
0.0009547602892453066
0.0009545693371874575
0.00095437842332002
0.000954187547635356
0.0009539967101258289
0.0009538059107838038
0.000953615149601647
0.0009534244265717268
0.0009532337416864124
0.0009530430949380752
0.0009528524863190876
0.0009526619158218237
0.0009524713834386593
0.0009522808891619715
0.000952090432984139
0.0009519000148975422
0.0009517096348945626
0.0009515192929675837
0.0009513289891089902
0.0009511387233111685
0.0009509484955665062
0.000950758305867392

0.0008486514170507664
0.0008484816867673562
0.0008483119904300027
0.0008481423280319167
0.0008479726995663104
0.0008478031050263971
0.0008476335444053918
0.0008474640176965107
0.0008472945248929714
0.0008471250659879927
0.0008469556409747952
0.0008467862498466002
0.0008466168925966308
0.0008464475692181115
0.0008462782797042679
0.0008461090240483271
0.0008459398022435174
0.0008457706142830687
0.0008456014601602121
0.0008454323398681801
0.0008452632534002064
0.0008450942007495264
0.0008449251819093765
0.0008447561968729947
0.00084458724563362
0.0008444183281844934
0.0008442494445188564
0.0008440805946299527
0.0008439117785110267
0.0008437429961553245
0.0008435742475560935
0.0008434055327065823
0.000843236851600041
0.0008430682042297209
0.0008428995905888749
0.0008427310106707571
0.000842562464468623
0.0008423939519757292
0.0008422254731853341
0.000842057028090697
0.0008418886166850788
0.0008417202389617418
0.0008415518949139494
0.0008413835845349667
0.0008412153078180596
0.0008410470647

In [6]:
# Тестирование перцептрона

# до активации нейронов preActiv[x][y]
# x - индекс слоя (слои считаются с первого скрытого слоя)
# y - индекс нейрона в слое
preActiv = np.zeros((len(neuron_count) - 1, max_neuron_count))
# после активациии нейронов postActiv[x][y]
# x - индекс слоя (слои считаются с входного слоя)
# y - индекс нейрона в слое
postActiv = np.zeros((len(neuron_count), max_neuron_count))
    
right = 0
y_real = len(test_data)
for i in range(len(test_data)):
   
    # получаем нормализованный входной вектор со слоя Кохонена
    # student = test_data[random.randint(0, len(train_data) - 1)]
    # train_el = getKohonenNormResult(student['features'], kohonen_neuron_count, kohonen_w)    

    # подаем на вход интересующее значение
    student = test_data[i]
    test_el = student['features']

    for i in range(len(test_el)):
        postActiv[0][i] = test_el[i]

    # Проходим по всем слоям
    for l in range(len(neuron_count) - 1):
        for n_c in range(neuron_count[l + 1]):
            preActiv[l][n_c] = np.dot(postActiv[l][0:neuron_count[l]], weights[l][0:neuron_count[l],n_c])
            postActiv[l+1][n_c] = activation(preActiv[l][n_c])    

    print(student['name'])
    print(student['result'])
    if (student['result'][0] == 1):
        y_real += 1
    print('Результаты нейросети:')
    result = postActiv[len(neuron_count)-1][0:len(student['result'])]
    print(str(result[0] * 100) + '%')
    pred = 1 if (result[0] * 100) > 50 else 0
        
    if (student['result'][0] == pred):
        right += 1
    
#     for i in range(len(result)):
#         print(str(result[i] * 100) + '%')
#         if (result[i] * 100) > 50:
#             y_pred += 1
        
    print()
    print('=================================================')
    print()
print('Result ' + str((right / y_real)*100))

Carter, Master. William Thornton II
[1]
Результаты нейросети:
16.518356286734274%


Thomas, Master. Assad Alexander
[1]
Результаты нейросети:
19.043399177322048%


Hedman, Mr. Oskar Arvid
[1]
Результаты нейросети:
16.417238087745726%


Johansson, Mr. Karl Johan
[0]
Результаты нейросети:
16.483256217312043%


Andrews, Mr. Thomas Jr
[0]
Результаты нейросети:
18.72364746551293%


Pettersson, Miss. Ellen Natalia
[0]
Результаты нейросети:
59.91859385559063%


Meyer, Mr. August
[0]
Результаты нейросети:
17.06974214102033%


Chambers, Mrs. Norman Campbell (Bertha Griggs)
[1]
Результаты нейросети:
80.42015786790904%


Alexander, Mr. William
[0]
Результаты нейросети:
16.40163119839618%


Lester, Mr. James
[0]
Результаты нейросети:
16.631563981066623%


Slemen, Mr. Richard James
[0]
Результаты нейросети:
16.92485652659583%


Andersson, Miss. Ebba Iris Alfrida
[0]
Результаты нейросети:
60.59416866947516%


Tomlin, Mr. Ernest Portage
[0]
Результаты нейросети:
16.474695650944852%


Fry, Mr. Richard